<a href="https://colab.research.google.com/github/Fran-Sierra/Data-analysis-in-Football/blob/main/colab_files/xT_data_creation(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tables==3.6.1
!pip install matplotsoccer

  Using cached tables-3.6.1.tar.gz (4.6 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tables
  Running setup.py clean for tables
Failed to build tables
ERROR: Could not build wheels for tables, which is required to install pyproject.toml-based projects


In [2]:
!pip install tables==3.6.1
!pip install socceraction==0.2.0

  Using cached tables-3.6.1.tar.gz (4.6 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tables
  Running setup.py clean for tables
Failed to build tables
ERROR: Could not build wheels for tables, which is required to install pyproject.toml-based projects


In [3]:
pip install scipy

In [4]:
pip install numpy==1.24.4

In [5]:
!pip install tables==3.6.1
!pip install matplotsoccer

  Using cached tables-3.6.1.tar.gz (4.6 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tables
  Running setup.py clean for tables
Failed to build tables
ERROR: Could not build wheels for tables, which is required to install pyproject.toml-based projects


In [6]:
pip install mplsoccer

In [7]:
!pip install matplotsoccer

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')
%mkdir -p '/content/gdrive/My Drive/Friends of Tracking/'
%cd '/content/gdrive/My Drive/Friends of Tracking/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Friends of Tracking


In [9]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import BSpline, make_interp_spline

import socceraction.xthreat as xthreat
import socceraction.spadl as spadl

In [10]:
import warnings

import matplotsoccer as mps
import numpy as np
import pandas as pd
from ipywidgets import fixed, interact_manual, widgets
from tqdm.notebook import tqdm

In [11]:
import warnings
from io import BytesIO
from pathlib import Path
from urllib.parse import urlparse
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile, is_zipfile

import pandas as pd
import socceraction.vaep.features as features
import socceraction.vaep.labels as labels
from sklearn.metrics import brier_score_loss, roc_auc_score
from socceraction.spadl.wyscout import convert_to_spadl
from socceraction.vaep.formula import value
from tqdm.notebook import tqdm
from xgboost import XGBClassifier

In [12]:
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

In [13]:
data_files = {
    'events': 'https://ndownloader.figshare.com/files/14464685',  # ZIP file containing one JSON file for each competition
    'matches': 'https://ndownloader.figshare.com/files/14464622',  # ZIP file containing one JSON file for each competition
    'players': 'https://ndownloader.figshare.com/files/15073721',  # JSON file
    'teams': 'https://ndownloader.figshare.com/files/15073697'  # JSON file
}

In [14]:
for url in tqdm(data_files.values()):
    url_s3 = urlopen(url).geturl()
    path = Path(urlparse(url_s3).path)
    file_name = path.name
    file_local, _ = urlretrieve(url_s3, file_name)
    if is_zipfile(file_local):
        with ZipFile(file_local) as zip_file:
            zip_file.extractall()

  0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
def read_json_file(filename):
    with open(filename, 'rb') as json_file:
        return BytesIO(json_file.read()).getvalue().decode('unicode_escape')

In [16]:
json_teams = read_json_file('teams.json')
df_teams = pd.read_json(json_teams)

In [17]:
df_teams.to_hdf('wyscout.h5', key='teams', mode='w')

In [18]:
json_players = read_json_file('players.json')
df_players = pd.read_json(json_players)

In [19]:
df_players.to_hdf('wyscout.h5', key='players', mode='a')

In [20]:
competitions = [
     'England',
#     'France',
#     'Germany',
#     'Italy',
    # 'Spain',
    # 'European Championship',
    # 'World Cup'
]

In [21]:
dfs_matches = []
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_matches = f'matches_{competition_name}.json'
    json_matches = read_json_file(file_matches)
    df_matches = pd.read_json(json_matches)
    dfs_matches.append(df_matches)
df_matches = pd.concat(dfs_matches)

In [22]:
df_matches.to_hdf('wyscout.h5', key='matches', mode='a')

In [23]:
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_events = f'events_{competition_name}.json'
    json_events = read_json_file(file_events)
    df_events = pd.read_json(json_events)
    df_events_matches = df_events.groupby('matchId', as_index=False)
    for match_id, df_events_match in df_events_matches:
        df_events_match.to_hdf('wyscout.h5', key=f'events/match_{match_id}', mode='a')

In [24]:
convert_to_spadl('wyscout.h5', 'spadl.h5')

...Inserting actiontypes
...Inserting bodyparts
...Inserting results
...Converting games
...Converting players
...Converting teams
...Generating player_games


100%|██████████| 380/380 [00:42<00:00,  8.98game/s]


...Converting events to actions


100%|██████████| 380/380 [04:41<00:00,  1.35game/s]


In [25]:
df_games = pd.read_hdf('spadl.h5', key='games')
df_actiontypes = pd.read_hdf('spadl.h5', key='actiontypes')
df_bodyparts = pd.read_hdf('spadl.h5', key='bodyparts')
df_results = pd.read_hdf('spadl.h5', key='results')
df_teams = pd.read_hdf('spadl.h5', key='teams')
df_players = pd.read_hdf('spadl.h5', key='players')

In [27]:
# team_name_mapping = df_teams.set_index('team_id')['team_name'].to_dict()
# df_games['home_team_name'] = df_games['home_team_id'].map(team_name_mapping)
# df_games['away_team_name'] = df_games['away_team_id'].map(team_name_mapping)

# Events for a single match


In [28]:
# Select a game of choice
game_id = 2499858

# Events for a single match
with pd.HDFStore('spadl.h5') as spadlstore:
    df_actions = spadlstore[f'actions/game_{game_id}']
    df_actions = (
        df_actions.merge(spadlstore['actiontypes'], how='left')
        .merge(spadlstore['results'], how='left')
        .merge(spadlstore['bodyparts'], how='left')
        .merge(spadlstore['players'], how="left")
        .merge(spadlstore['teams'], how='left')
        .reset_index()
        .rename(columns={'index': 'action_id'})
    )
df_actions.to_csv(f'/content/gdrive/MyDrive/Colab Notebooks/files/project/actions_{game_id}.csv')

#Events for each match of a particular team (only events for the chosen team)

In [29]:
# select a team of choice (if you want all matches for a particular team)
team=1644

team_matches=[]
for i, row in df_games.iterrows():
  if row['home_team_id']==team or row['away_team_id']==team:
    team_matches.append(row['game_id'])

matches=pd.Series(team_matches)

matches.to_csv(f'/content/gdrive/MyDrive/Colab Notebooks/files/project/matches_{team}.csv')

# Events for each match of a particular team (only events for the chosen team)
df=pd.DataFrame()
for game_id in team_matches:
  with pd.HDFStore('spadl.h5') as spadlstore:
    df_actions = spadlstore[f'actions/game_{game_id}']
    df_actions = (
        df_actions.merge(spadlstore['actiontypes'], how='left')
        .merge(spadlstore['results'], how='left')
        .merge(spadlstore['bodyparts'], how='left')
        .merge(spadlstore['players'], how="left")
        .merge(spadlstore['teams'], how='left')
        .reset_index()
        .rename(columns={'index': 'action_id'})
    )
    df_actions_team=df_actions[df_actions.team_id==team]
    df_actions_team.to_csv(f'/content/gdrive/MyDrive/Colab Notebooks/files/project/actions_{team}_{game_id}.csv')



# EVENTS FOR BOTH TEAMS IN THE MATCH (Only matches of the chosen team, but events for both teams in the match)


In [31]:
# select a team of choice (if you want all matches for a particular team)
team=1613

team_matches=[]
for i, row in df_games.iterrows():
  if row['home_team_id']==team or row['away_team_id']==team:
    team_matches.append(row['game_id'])

matches=pd.Series(team_matches)

matches.to_csv(f'/content/gdrive/MyDrive/Colab Notebooks/files/project/matches_{team}.csv')

# EVENTS FOR BOTH TEAMS IN THE MATCH (Only matches of the chosen team, but events for both teams in the match)

df=pd.DataFrame()
for game_id in team_matches:
  with pd.HDFStore('spadl.h5') as spadlstore:
    df_actions = spadlstore[f'actions/game_{game_id}']
    df_actions = (
        df_actions.merge(spadlstore['actiontypes'], how='left')
        .merge(spadlstore['results'], how='left')
        .merge(spadlstore['bodyparts'], how='left')
        .merge(spadlstore['players'], how="left")
        .merge(spadlstore['teams'], how='left')
        .reset_index()
        .rename(columns={'index': 'action_id'})
    )
    df_actions_match=df_actions
    df_actions_match.to_csv(f'/content/gdrive/MyDrive/Colab Notebooks/files/project/actions_{game_id}.csv')
